In [1]:
import tensorflow as tf

In [2]:
tf.enable_eager_execution()
tf.executing_eagerly()

True

In [8]:
import numpy as np
from itertools import product

In [13]:
n = 100
x = np.linspace(-1, 1, n)
y = np.linspace(-1, 1, n)
xv, yv = np.meshgrid(x, y)
inputs = tf.expand_dims(tf.constant(list(product(x, y)), dtype=np.float32), dim=1)

w1 = tf.Variable(np.random.normal(size=[2, 2]).astype(np.float32))
w2 = tf.Variable(np.random.normal(size=[2, 1]).astype(np.float32))
# w3 = tf.Variable(np.random.normal(size=[,10]).astype(np.float32))


In [35]:
class NLN(tf.keras.Model):
    def __init__(self, w1, w2):
        super().__init__()
        self.w1 = w1
        self.w2 = w2
    
    def call(self, x):
        h1 = tf.nn.relu(tf.nn.bias_add((x, self.w1), 0))
        output = tf.relu(tf.matmul(h1, self.w2))
        return output

In [36]:
class NLLN(tf.keras.Model):
    def __init__(self, w1, w2, scale=2**125):
        super().__init__()
        self.w1 = w1
        self.w2 = w2
        self.scale = scale
    
    def call(self, x):
        h1 = (tf.matmul(x, self.w1) / self.scale)
        output = ((tf.matmul(h1, self.w2)) / self.scale) * self.scale
        return output

In [37]:
nln = NLN(w1, w2)
nlln = NLLN(w1, w2)

In [38]:
nln_outputs = nln(inputs)
nlln_outputs = nlln(inputs)

InvalidArgumentError: Shapes of all inputs must match: values[0].shape = [10000,1,2] != values[1].shape = [2,2] [Op:Pack] name: packed

In [22]:
inputs[0]

<tf.Tensor: id=39, shape=(1, 2), dtype=float32, numpy=array([[-1., -1.]], dtype=float32)>